<a href="https://colab.research.google.com/github/restiso7788/COVID-19-spread-trend/blob/master/LM_Logistic_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#import exper data 
url = "https://raw.githubusercontent.com/sanderlab/CellBox/master/data/expr.csv"
url_exp_node = "https://raw.githubusercontent.com/sanderlab/CellBox/master/data/node_Index.csv"
node = pd.read_csv(url_exp_node, header= None)
node = node.transpose()
df = pd.read_csv(url, header= None)

# 89 perturbation conditions; 
# 99 components connecting perturbations, protein response, and phenotypes to simulate dynamic cellular behaviour

In [ ]:
# append the node as the exper column name
name = node.values.tolist()
df.columns = name
df.shape
df.head()

,4EBP1pS65,RbpS807,MAPKpT202,MEKpS217,S6,PAI-1,AKTpS473,AMPKpT172,b-Catenin,BIM,Caveolin,CyclinB1,CyclinD1,GSK3abpS21,GSKpS9,IGFBP2,p38pT180,p53,p70S6KpT389,SRCpY527,STAT3pY705,TSC2pT1462,YAPpS127,ACC1,AKTpT308,AKT,b-CateninpS,Fibronectin,HSP27,IGF1R-beta,IRS1,p27,c-Myc,SMAD3,STAT3,STAT5,STAT5pY694,COX2,PAX2,PLK1,...,ERa,GATA3,GSK3ab,mTORpS2448,p21,STAT6pY641,TAZpS89,XRCC1,PCNA,PDK1pS241,PI3Kp85,PKCa,RAD51,SRCpY416,4EBP1pT70,ACCpS79,Collagenase,CyclinE1,IRS1pS307,p38,SMAD3pS423,TSC2,SRC,G2M,G1arrest,G2arrest,Sarrest,cellviab,aMEK,aAKT,aHDAC,aMDM2,aJAK,aBRAFm,aPKC,aSTAT3,amTOR,aPI3K,aCDK4,aSRC
0,-0.435,-0.608,-0.855,-0.712,-0.113,-0.089,0.096,0.291,0.428,0.375,0.238,-0.338,-0.559,-0.614,-0.514,0.048,-0.298,0.277,-0.626,-0.306,0.131,-0.283,-0.089,0.056,0.044,-0.333,-0.006,-0.024,0.049,0.022,-0.045,0.315,-0.111,0.032,-0.024,0.116,-0.066,-0.367,0.297,-0.022,...,0.281,0.135,0.040,-0.159,-0.042,0.141,0.092,0.154,0.177,0.103,0.090,0.277,-0.121,-0.076,0.045,0.305,0.201,0.052,0.047,-0.004,0.132,0.024,0.182,0.0,0.563,0.000,0.0,0.242,-0.698,0.0,0.00,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.308,-0.322,-0.999,-0.594,0.022,-0.061,0.093,0.222,0.255,0.645,-0.420,-0.114,-0.564,-0.520,-0.438,0.391,-0.202,-0.172,-0.585,-0.154,0.000,-0.274,-0.058,-0.083,0.159,-0.273,-0.002,0.173,0.055,0.027,0.166,0.436,-0.312,-0.118,0.405,-0.173,0.001,0.743,0.195,-0.007,...,0.210,-0.042,0.104,-0.116,-0.038,0.090,0.088,0.041,0.056,0.300,0.053,0.147,-0.223,0.121,0.150,0.210,0.081,-0.051,-0.074,-0.015,0.239,-0.024,0.049,0.0,0.000,1.748,0.0,0.431,-0.698,0.0,-0.89,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.402,-0.549,-0.837,-0.675,-0.011,-0.103,0.089,0.418,0.276,0.439,0.084,-0.265,-0.569,-0.500,-0.505,0.092,-0.176,0.343,-0.653,-0.281,0.102,-0.265,0.123,0.100,0.087,-0.147,-0.025,0.002,0.091,0.134,0.017,0.445,-0.255,-0.023,-0.054,0.065,0.009,-0.854,0.173,-0.071,...,0.195,-0.010,0.090,-0.192,-0.091,0.118,0.046,0.028,0.025,0.124,0.066,0.218,-0.198,0.073,0.112,0.307,0.095,0.006,-0.059,0.023,0.109,0.021,0.230,0.0,0.464,0.000,0.0,0.143,-0.698,0.0,0.00,-0.393,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.475,-0.652,-0.935,-0.751,-0.085,-0.019,0.064,0.207,0.147,0.416,0.170,-0.390,-0.541,-0.740,-0.624,0.174,-0.151,0.237,-0.631,-0.289,-0.023,-0.342,-0.049,0.026,-0.080,-0.266,-0.049,0.065,0.081,0.149,0.028,0.410,-0.267,-0.066,-0.080,-0.004,-0.028,-0.509,0.198,-0.148,...,0.175,0.035,-0.013,-0.147,-0.063,0.120,0.063,0.021,0.188,0.098,-0.032,0.183,-0.176,0.115,0.017,0.186,0.120,-0.042,-0.228,-0.068,0.072,-0.024,0.221,0.0,0.532,0.000,0.0,-0.070,-0.698,0.0,0.00,0.000,-0.474,0.000,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.611,-1.097,-1.234,-0.928,-0.114,-0.113,-0.002,-0.042,0.277,0.732,0.205,-0.799,-0.829,-0.758,-0.660,0.187,-0.214,0.316,-0.825,-0.236,0.154,-0.337,0.021,0.020,0.010,-0.212,-0.027,0.096,0.154,0.200,-0.127,0.714,-0.409,-0.013,-0.064,0.005,0.020,-0.789,0.256,-0.299,...,0.238,0.016,0.052,-0.210,-0.093,0.116,0.109,0.114,0.120,0.128,0.032,0.236,-0.301,0.052,-0.083,0.174,0.189,0.025,-0.156,-0.048,0.286,0.037,0.282,0.0,0.701,0.000,0.0,0.286,-0.698,0.0,0.00,0.000,0.000,-0.552,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# specify X(D in our document concentration of the drugs were applies) and Y(X in the document)
x = df.iloc[:, -12:].values
y1 = df.iloc[:,0].values

In [ ]:
# creat training and testing set(70% for training, n = 62 conditions), with same proportation as CellBox
X_train, X_test, y_train, y_test = train_test_split(x, y1, test_size=0.3)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(62, 12) (62,)
(27, 12) (27,)


In [ ]:
# fit a model(Linear regression for k =1(the first column as the reponse from the data))
from sklearn.linear_model import LinearRegression
lm = LinearRegression().fit(X_train, y_train)
print(lm.intercept_)
print(lm.coef_)

-0.16316168140522214
[ 0.28537473  0.50775274  0.11689682  0.24836658 -0.04537315  0.0449108
  0.46546282  0.51891533  0.27134484  0.58630446  1.78113703  0.19274151]


In [ ]:
# predictions on test data 
y_pred = lm.predict(X_test)

Future update, 
1. estimate the parameters for each protein, here is just the example for K =1
2. will summarize every model performance by using coreelation of predicitons, minic the figure 2B and 2C from CellBox

3. will check the difference betweent he exper and pert data, looks like the the CellBox paper use the perturbation data to train their model, but all response are zero, so still confuse. 
